## Finetuning YOLOv11 with Buffalo and Camel labels.
For installation and setup, view the [README](./README.md) before running this.

As mentioned in the README, we will be using Roboflow for labelling our dataset.

The Roboflow project is [available here](https://app.roboflow.com/stuff-avvl2/yolo-fine-y444l).

## Download Annotated Dataset
We will download the versioned dataset directly from Roboflow to the target directory.

v8 format is used as it is widely supported.

In [12]:
import os
from roboflow import Roboflow
from tqdm import tqdm
from dotenv import load_dotenv
from pathlib import Path

load_dotenv()
target_path = Path("yolo-fine-1/")

if target_path.exists() and len(os.listdir(target_path)) > 5:
    print(f"Dataset already found at: {target_path}. Skipping download.")
else:
    API_KEY = os.getenv("ROBOFLOW_API_KEY")
    rf = Roboflow(api_key=API_KEY)
    project = rf.workspace("stuff-avvl2").project("yolo-fine-y444l")
    dataset = project.version("1").download("yolov8")

Dataset already found at: yolo-fine-1. Skipping download.


## Training the Model

### Setup the logging

Ultralytics support logging to wandb, comet.ml and tensorboard, out of the box. Here we only enable wandb.

You need to create an account at [wandb](https://wandb.ai) and get the API key from https://wandb.ai/authorize.

In [1]:
from ultralytics import settings

settings.update({"wandb": True,
                 "tensorboard": False})

### Training

We specify the path to data.yaml file, and train with a batch size of 15, and we also save the checkpoint at each epoch (save_period=1). We assume here you are connected to a GPU, hence we can specify the device to use as `device=0` to select the first GPU.  We specify the project name as balloon, this will create a folder called `balloon` to store the weights and various training artifacts such as F1, PR curves, confusion matrics, training results (loss, mAP, etc).

For a complete listing of train settings, you can see [here](https://docs.ultralytics.com/modes/train/#train-settings).

You can also specify the type of data [augmentation](https://docs.ultralytics.com/modes/train/#augmentation-settings-and-hyperparameters)  you want as part of the train pipeline.

You can monitor your training progress at wandb (the link is given in the train output below)

My GPU specs are not the best, hence we will defer training logic to the terminal via a [Python script](train_model.py) to prevent out of memory issues.
> Even if you have good hardware, please do not run without workers set to a low value or the Jupyter kernel might crash.

Below is the applied training logic.

In [1]:
# from ultralytics import YOLO
# import torch

# model = YOLO("yolo11s.pt")

# result = model.train(data="yolo-fine-1/data.yaml",
#                      epochs=30,
#                      save_period=1,
#                      batch=8,
#                      workers=2,
#                      device="0",
#                      project='yolo-fine',
#                      plots=True
#                     )

After training we view the metrics on WanDB.

We can run the best model (using the best checkpoint) against the validation dataset to see the overall model performance on validation set.

<!-- You should see around 0.88 for mAP50, and 0.78 for mAP50-95. -->

In [7]:
from ultralytics import YOLO

model = YOLO("yolo-fine/train/weights/best.pt")
validation_results = model.val(data="yolo-fine-1/data.yaml", device="0")

print("\n--- Key Metrics ---")
print(f"Overall mAP50 (map50): {validation_results.box.map50:.4f}")
print(f"Overall mAP50-95 (map): {validation_results.box.map:.4f}")
print(f"Mean Precision (mp): {validation_results.box.mp:.4f}")
print(f"Mean Recall (mr): {validation_results.box.mr:.4f}")

Ultralytics 8.3.235 🚀 Python-3.10.19 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 2050, 3771MiB)
YOLO11s summary (fused): 100 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1558.3±926.0 MB/s, size: 65.2 KB)
val: Scanning /home/data/Desktop/projects/yolo-finetune/yolo-fine-1/valid/labels.cache... 164 images, 2 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 164/164 177.2Kit/s 0.0s
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 182, len(boxes) = 308. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 11/11 3.2it/s 3.4s0.3s
                   all        164        308      0.861      0.779      0.851      0.648
               buffalo         76        158      0.879      0.778       